In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [0]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv2D, MaxPooling2D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.neural_network import MLPClassifier
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re

In [0]:
fields = ['Event 1', 'Event 2', 'Label']

df_train = pd.read_csv('P1_training_set.csv', skipinitialspace=True, usecols=fields)
df_test = pd.read_csv('P1_testing_set.csv', skipinitialspace=True, usecols=fields)

In [0]:
from nltk.stem.snowball import SnowballStemmer
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    
    return text
  
# apply the above function to df['text']
df_train['Event 1'] = df_train['Event 1'].map(lambda x: clean_text(x))
df_train['Event 2'] = df_train['Event 2'].map(lambda x: clean_text(x))

df_test['Event 1'] = df_test['Event 1'].map(lambda x: clean_text(x))
df_test['Event 2'] = df_test['Event 2'].map(lambda x: clean_text(x))

In [0]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_sm')
# process a sentence using the model
sent1_train = []
for sentence in df_train['Event 1'].astype(str):
  sent1_train.append(nlp(sentence))

sent2_train = []
for sentence in df_train['Event 2'].astype(str):
  sent2_train.append(nlp(sentence))
  
sent1_test = []
for sentence in df_test['Event 1'].astype(str):
  sent1_test.append(nlp(sentence))

sent2_test = []
for sentence in df_test['Event 2'].astype(str):
  sent2_test.append(nlp(sentence))

In [0]:
data_train = []
data_test = []
for i in range(len(sent1_train)):
  data_train.append(np.concatenate([sent1_train[i].vector, sent2_train[i].vector]))
for i in range(len(sent1_test)):
  data_test.append(np.concatenate([sent1_test[i].vector, sent2_test[i].vector]))

data_train = np.array(data_train)
data_test = np.array(data_test)

In [49]:
## Network architecture
labels = df_train['Label']

model_spacy = MLPClassifier(solver='adam', max_iter=100, alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
model_spacy.fit(data_train, labels)

model_spacy.score(data_test, df_test['Label'])

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.



0.4990700557966522